# EAST Detector 

In [25]:
import imutils
import cv2
import time

In [2]:
# load the image and check it's correct loaded 
im1 = cv2.imread("data/spa/spa1_01MO.jpg")
cv2.imshow('Image 1', im1)
cv2.waitKey()
cv2.destroyAllWindows()

#### EAST detector only allows multiples of 32

In [17]:
# predefine width and height
im_w = 320 
im_h = 320 

# Create a copy of the image 
im1_c = im1.copy()
H, W, _ = im1_c.shape

# Calculate the ratio in change for both directions
r_W = H / im_w
r_H = W / im_w

# Resize the image to the prespecified dimensions 
im1_r = cv2.resize(im1_c, (im_w, im_h))

In [18]:
# Check the new image 
cv2.imshow('Image 1', im1_r)
cv2.waitKey()
cv2.destroyAllWindows()

Try to decrease a lot the image and see if the detection speed improves. We have to iterate through all the columns and rows to detect changes in intensity

## Load the EAST algorithm

In [26]:
layersNames = [
    "feature_fusion/Conv_7/Sigmoid",  # Layer to out the probabilities (Confidence of the text box)
    "feature_fusion/concat_3"         # Derive the bounding box coordinates (Geometry of the text box)
]

In [23]:
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

In [24]:
blob = cv2.dnn.blobFromImage(im1_r, 1.0, (im_w, im_h), (123.68, 116.78, 103.94), True, False)
# The first argument is the image itself
# The second argument specifies the scaling of each pixel value. Not required, keep it 1.
# The default input to the network is 320×320. So, we need to specify this while creating the blob.
# We also specify the mean that should be subtracted from each image, this was used while training the model
# The mean used is (123.68, 116.78, 103.94).
# The next argument is whether we want to swap the R and B channels. This is required since OpenCV uses BGR 
# format and Tensorflow uses RGB format.
# The last argument is whether we want to crop the image and take the center crop. 

In [27]:
# Track time 
start = time.time()
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
net.setInput(blob)
scores, geometry = net.forward(layersNames)
end = time.time()

In [29]:
print(f"Average time {end-start}")

Average time 0.2644991874694824


In [31]:
scores.shape

(1, 1, 80, 80)

In [ ]:
[boxes, confidences] = decode(scores, geometry, confThreshold)
